<a href="https://colab.research.google.com/github/DavoodSZ1993/Dive_into_Deep_Learning/blob/main/6_builder_guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 6.1 Layers and Modules

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [8]:
net = nn.Sequential(nn.LazyLinear(256),   # input x 256
                    nn.ReLU(),            
                    nn.LazyLinear(10))    # 256 x 10

X = torch.rand(2, 20) # two samples (rows) & 20 features (columns)
X.shape, net(X).shape

(torch.Size([2, 20]), torch.Size([2, 10]))

### 6.1.1 A Custom Module